## Install relevant packages

In [3]:
!pip install bitsandbytes faiss-cpu sentence-transformers datasets

In [4]:
MODEL_NAME = "meta-llama/few-shot"
CHECKPOINT_PATH = f'{MODEL_NAME}'

In [5]:
import os
def create_directory_if_not_exists(dir_path):
    # Check if the directory exists
    if not os.path.exists(dir_path):
        # Create the directory (and any necessary parent directories)
        os.makedirs(dir_path)
        print(f"Directory '{dir_path}' created.")
    else:
        print(f"Directory '{dir_path}' already exists.")
        
create_directory_if_not_exists(CHECKPOINT_PATH)

Directory 'meta-llama/few-shot' created.


## Import all relevant libraries

In [5]:
import os
from huggingface_hub import login
import datasets
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import pandas as pd
import re

In [ ]:
os.environ["HUGGINGFACE_TOKEN"] = ""
login(os.environ["HUGGINGFACE_TOKEN"])

In [7]:
dataset = "chengxuphd/liar2"
dataset = datasets.load_dataset(dataset)

README.md:   0%|          | 0.00/2.87k [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/19.0M [00:00<?, ?B/s]

valid.csv:   0%|          | 0.00/2.38M [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/2.38M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/18369 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2297 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2296 [00:00<?, ? examples/s]

In [8]:
# Convert labels: {0, 1, 2, 3} -> 1 (aka fake), {4, 5} -> 0 (aka real)
def convert_labels(data):
    if data["label"] in [0, 1, 2, 3]:
        data["true_label"] = 1
    else:
        data["true_label"] = 0
    return data

In [9]:
dataset = dataset.map(convert_labels)
test_dataset = dataset["test"]

Map:   0%|          | 0/18369 [00:00<?, ? examples/s]

Map:   0%|          | 0/2297 [00:00<?, ? examples/s]

Map:   0%|          | 0/2296 [00:00<?, ? examples/s]

In [10]:
# Load embedding model
embedding_model = SentenceTransformer("all-mpnet-base-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

### Compute embeddings for justifications

In [11]:
# Extract justifications
justifications = [sample["justification"] for sample in test_dataset]

# Convert justifications into embeddings
justification_embeddings = embedding_model.encode(justifications, convert_to_numpy=True)

# Create FAISS index
dimension = justification_embeddings.shape[1]  # Get embedding size
faiss_index = faiss.IndexFlatL2(dimension)  # L2 (Euclidean) distance
faiss_index.add(justification_embeddings)   # Store embeddings

Batches:   0%|          | 0/72 [00:00<?, ?it/s]

In [12]:
def retrieve_top_k_justifications(claim, k=2):
    """Retrieves top-k relevant justifications using FAISS"""
    
    # Convert claim into embedding
    claim_embedding = embedding_model.encode([claim], convert_to_numpy=True)
    
    # Retrieve top-k justifications
    distances, indices = faiss_index.search(claim_embedding, k)
    
    # Return retrieved justifications
    return [justifications[i] for i in indices[0]], indices[0]

In [13]:
def format_chat_prompt_v1(claim, speaker, speaker_description):
    """Formats the input prompt using retrieved justifications as context. (zero-shot)"""

    # Retrieve relevant justifications
    retrieved_justifications, indices = retrieve_top_k_justifications(claim, k=2)
    justification_text = "\n\n".join(retrieved_justifications)  # Combine into one text block
    indices = indices.tolist()
    
    chat_prompt = tokenizer.apply_chat_template(
        [
            {"role": "system", "content": "You are an AI that detects fake news using both linguistic analysis and contextual information retrieved from a knowledge base."},

            # Pass retrieved justifications as context
            {"role": "user", "content": f"""Here is the relevant information from the knowledge base:\n\n"{justification_text}"

Classify the following statement as FAKE or REAL, and highlight only the top five single keywords contributing to the classification.

Focus on both linguistic features and the given contextual evidence.

Statement: "{claim}" 

Speaker: "{speaker}"

Speaker Description: "{speaker_description}"
            
Output the result in the following structured format:
    
Classification: (FAKE or REAL)
    
Keywords: list of ONLY five single words without explanation, in the format: [kw1, kw2, kw3, kw4, kw5]
    
Reasoning: (Explain based on both linguistic features and the given contextual evidence)"""}
        ],
        tokenize=False,
        add_generation_prompt=True,
    )

    return chat_prompt, justification_text, indices

In [14]:
def format_chat_prompt_v2(claim, speaker, speaker_description):
    """Formats the input prompt using retrieved justifications as context. (two-shot)"""

    # Retrieve relevant justifications
    retrieved_justifications, indices = retrieve_top_k_justifications(claim, k=2)
    justification_text = "\n\n".join(retrieved_justifications)  # Combine into one text block
    indices = indices.tolist()
    
    chat_prompt = tokenizer.apply_chat_template(
        [
            {"role": "system", "content": "You are an AI that detects fake news using both linguistic analysis and contextual information retrieved from a knowledge base."},

            # Example 1 (FAKE)
            {"role": "user", "content": f"""Here is the relevant information from the knowledge base:

"According to CBS News White House reporter Mark Knoller, who keeps records of presidential travel, President Obama took 29 vacation trips over his eight years in the White House. The trips spanned all or part of 235 days (not 692). He also made 39 visits to Camp David, totaling around 93 days. Even though the original story was published with a disclaimer, versions of the account were copied and pasted in social media posts with no such explanation and circulated as real news.  We rate this !"

Classify the following statement as FAKE or REAL, and highlight only the top five single keywords contributing to the classification.

Focus on both linguistic features and the given contextual evidence.

Statement: "President Obama used government funds to pay for his personal vacations for the next 20 years and Trump sent him a bill." 

Speaker: "facebook posts"

Speaker Description: "Posters on Facebook and other social media networks."
            
Output the result in the following structured format:
    
Classification: (FAKE or REAL)
    
Keywords: list of ONLY five single words without explanation, in the format: [kw1, kw2, kw3, kw4, kw5]
    
Reasoning: (Explain based on both linguistic features and the given contextual evidence)"""},

            {"role": "assistant", "content": """Classification: FAKE
            
Keywords: Obama, vacations, government, bill, Trump
            
Reasoning: Linguistically, the statement uses sensational and emotionally charged language (“used government funds,” “for the next 20 years,” “Trump sent him a bill”) common in fake news. The exaggerated future timeline and improbable scenario (a former president receiving a bill from a sitting or later president) lacks credibility and factual basis.

From the contextual evidence, it is confirmed that President Obama took 29 vacation trips over eight years, totaling 235 days, and not 692 as claimed in viral posts. There is no verified report of misuse of government funds for future vacations or any bill sent by Trump. The original false claim was widely circulated on social media without context, further supporting that this is misinformation."""},

            # Example 2 (REAL)
            {"role": "user", "content": f"""Here is the relevant information from the knowledge base:

"Here is the relevant information from the knowledge base:
In sum, Graves’ numbers are correct. The VA’s budget has increased by more than 40 percent, and the number of pending claims has skyrocketed from some 391,000 to 890,000 under the Obama administration. Graves also was accurate in noting that the increase took place under the president’s watch. While the VA’s benefits system has been troubled since well before Obama took office, efforts during his administration have not prepared the department for its current challenges. We rate Graves’ statement ."

Classify the following statement as FAKE or REAL, and highlight only the top five single keywords contributing to the classification.

Focus on both linguistic features and the given contextual evidence.

Statement: "Despite having their budget increased by over 40 percent since 2009 … pending claims for benefits with the (Department of Veterans Affairs) have increased from 391,000 to 890,000 under the Obama Administration." 

Speaker: "tom graves"

Speaker Description: "Tom Graves represents Georgia's 9th Congressional District."
            
Output the result in the following structured format:
    
Classification: (FAKE or REAL)
    
Keywords: list of ONLY five single words without explanation, in the format: [kw1, kw2, kw3, kw4, kw5]
    
Reasoning: (Explain based on both linguistic features and the given contextual evidence)"""},

            {"role": "assistant", "content": """Classification: REAL
            
Keywords: budget, increased, claims, benefits, Obama
            
Reasoning: Linguistically, the statement is factual in tone, presenting specific numerical data without exaggeration or emotionally charged language. The use of figures and administrative references (e.g., "since 2009," "Department of Veterans Affairs") suggests a grounded, data-driven claim.

From the contextual evidence, the knowledge base confirms that the VA’s budget increased by over 40 percent under the Obama administration and that pending claims rose from approximately 391,000 to 890,000 during the same period. It also notes that although the issues existed before Obama’s tenure, the surge in claims and budget increase indeed occurred “under the president’s watch.” Therefore, the statement made by Tom Graves is consistent with the verified information."""},

            # Pass retrieved justifications as context
            {"role": "user", "content": f"""Here is the relevant information from the knowledge base:\n\n"{justification_text}"

Classify the following statement as FAKE or REAL, and highlight only the top five single keywords contributing to the classification.

Focus on both linguistic features and the given contextual evidence.

Statement: "{claim}" 

Speaker: "{speaker}"

Speaker Description: "{speaker_description}"
            
Output the result in the following structured format:
    
Classification: (FAKE or REAL)
    
Keywords: list of ONLY five single words without explanation, in the format: [kw1, kw2, kw3, kw4, kw5]
    
Reasoning: (Explain based on both linguistic features and the given contextual evidence)"""}
        ],
        tokenize=False,
        add_generation_prompt=True,
    )

    return chat_prompt, justification_text, indices

In [15]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

# Configure 4-bit quantization
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

# Use instruction-tuned models

model_name = "meta-llama/Llama-2-7b-chat-hf"
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quantization_config,
    device_map="auto"
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)



config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [16]:
def extract_prediction(response):
    """Extracts classification, keywords, and reasoning from model output."""
    
    # Extract classification (FAKE or REAL)
    classification_match = re.search(r'Classification:\s*(FAKE|REAL)', response)
    classification = classification_match.group(1) if classification_match else "UNKNOWN"

    # Extract keywords as a list
    # print(response)

    match = re.search(r"Keywords:\s*(.+?)\n", response)
    keywords = []
    if match:
        keywords = [kw.strip() for kw in match.group(1).split(',')]
    else:
        keywords = []
        
    # keyword_section = re.search(r"Keywords:\n(.*?)\n\n", response, re.DOTALL)
    # keywords = []
    # if keyword_section:
    #     lines = keyword_section.group(1).strip().split('\n')
    #     for line in lines:
    #         # Match patterns like: 1. "Keyword" - explanation OR 1. Keyword
    #         match = re.match(r'\d+\.\s*"?([^"-]+?)"?\s*(?:-|$)', line.strip())
    #         if match:
    #             keyword = match.group(1).strip()
    #             keywords.append(keyword)
    # else:
    #     keywords = []

    # Extract linguistic reasoning
    reasoning_match = re.search(r'Reasoning:\s*(.*)', response, re.DOTALL)
    reasoning = reasoning_match.group(1).strip() if reasoning_match else "No reasoning provided."
    
    return classification, keywords, reasoning

In [17]:
true_labels = []
pred_labels = []
results = []
i = 0

In [18]:
import pickle
with open(f'{CHECKPOINT_PATH}/true_labels.pkl', 'rb') as file:
    true_labels = pickle.load(file)

with open(f'{CHECKPOINT_PATH}/pred_labels.pkl', 'rb') as file:
    pred_labels = pickle.load(file)

with open(f'{CHECKPOINT_PATH}/results.pkl', 'rb') as file:
    results = pickle.load(file)

i = len(true_labels)
print(i)

1142


In [19]:
import pickle

"""Performs evaluation to extract results from test dataset into a .csv file and output evaluation metric scores."""
def evaluate(i, true_labels, pred_labels, results, shot):
    unknown_count = 0
    
    for sample in test_dataset.select(range(i, len(test_dataset))):
        if i % 50 == 0:
            print(f'{i} done')
            with open(f'{CHECKPOINT_PATH}/true_labels.pkl', 'wb') as file:
                pickle.dump(true_labels, file)
            
            with open(f'{CHECKPOINT_PATH}/pred_labels.pkl', 'wb') as file:
                pickle.dump(pred_labels, file)
    
            with open(f'{CHECKPOINT_PATH}/results.pkl', 'wb') as file:
                pickle.dump(results, file)

        i += 1
        
        unique_id = sample["id"]
        claim = sample["statement"]
        speaker = sample["speaker"]
        speaker_description = sample["speaker_description"]
        true_label = sample["true_label"]
        orig_label = sample["label"]
    
        # Format prompt using chat template
        if shot == 0: # zero-shot
            formatted_prompt, justification_text, indices = format_chat_prompt_v1(claim, speaker, speaker_description)
        else: # few-shot
            formatted_prompt, justification_text, indices = format_chat_prompt_v2(claim, speaker, speaker_description)
    
        # Tokenize and generate response
        inputs = tokenizer(formatted_prompt, return_tensors="pt").to("cuda")
        output_tokens = model.generate(**inputs, max_new_tokens=250)
        model_response = tokenizer.decode(output_tokens[0], skip_special_tokens=True)
    
        cleaned_response = model_response.split("[/INST]")[-1].strip()
    
        # Extract structured outputs
        predicted_label, keywords, reasoning = extract_prediction(cleaned_response)
        # print(keywords)
    
        # Convert predicted label to binary (FAKE=1, REAL=0)
        if predicted_label == "FAKE":
            pred_label = 1
            true_labels.append(true_label)
            pred_labels.append(pred_label)
        elif predicted_label == "REAL":
            pred_label = 0
            true_labels.append(true_label)
            pred_labels.append(pred_label)
        else: # Unknown label predicted
            # print(predicted_label)
            pred_label = -1
            unknown_count += 1
        
        results.append({
                "id": unique_id,
                "Claim": claim,
                "Retrieved Justification": justification_text,
                "Retrieved Index": indices,
                "Prediction": pred_label,
                "Keywords": ", ".join(keywords),
                "Reasoning": reasoning,
                "True Label": true_label,
                "Original Label": orig_label
            })
    
    df = pd.DataFrame(results)
    df.to_csv(f'{CHECKPOINT_PATH}/results.csv', index=False)
    
    # Compute evaluation metrics
    accuracy = accuracy_score(true_labels, pred_labels)
    precision, recall, f1, _ = precision_recall_fscore_support(true_labels, pred_labels, average="binary")
    return accuracy, precision, recall, f1, unknown_count

In [20]:
accuracy, precision, recall, f1, unknown_count = evaluate(i, true_labels, pred_labels, results, shot=1) # Few-shot prompt template

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")
print(f"Number of test instances where model outputs an unknown classification: {unknown_count}")
print(f"% of test instances where model outputs either FAKE or REAL: {(len(test_dataset) - unknown_count) / len(test_dataset)*100:.4f}")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Accuracy: 0.7339
Precision: 0.7752
Recall: 0.9000
F1-score: 0.8330
Number of test instances where model outputs an unknown classification: 4
% of test instances where model outputs either FAKE or REAL: 99.8258


In [21]:
with open(f'{CHECKPOINT_PATH}/true_labels.pkl', 'wb') as file:
    pickle.dump(true_labels, file)
            
with open(f'{CHECKPOINT_PATH}/pred_labels.pkl', 'wb') as file:
    pickle.dump(pred_labels, file)
    
with open(f'{CHECKPOINT_PATH}/results.pkl', 'wb') as file:
    pickle.dump(results, file)

In [22]:
print(accuracy, precision, recall, f1)

0.7338568935427574 0.7752293577981652 0.9 0.8329682365826945


RAG:
**Zero shot:**
`accuracy, precision, recall, f1: 0.2787456445993031 0.975 0.023022432113341203 0.04498269896193771`

**Few shot:**
`0.7338568935427574 0.7752293577981652 0.9 0.8329682365826945`

## Evaluation Scores
### Retrieval (Accuracy)
#### k=1
IndexFlatL2: 2073/2296 = 0.8649
#### k=2
**IndexFlatL2** & IndexFlatIP: 2073/2296 = 0.9029

IndexHNSWFlat: 2061/2296 = 0.8976

### RAG V1 (zero-shot)
Accuracy: 0.7629

Precision: 0.7311

Recall: 0.9350

F1-score: 0.8206

### RAG V2 (zero-shot) (swap label 3 to 1 (fake))
Accuracy: **0.8359**

Precision: **0.8725**

Recall: 0.9100 (Decreased likely because there are a lot more true fakes)

F1-score: **0.8908**

## Debugging Purposes

In [ ]:
# for i in range(10):
#     test_claim = test_dataset["statement"][i]
#     test_speaker = test_dataset["speaker"][i]
#     test_desc = test_dataset["speaker_description"][i]
#     test_label = test_dataset["true_label"][i]
#     # print(f"Claim:\n{train_claim}")
#     formatted_prompt, _, _ = format_chat_prompt_v2(test_claim, test_speaker, test_desc)
#     # print(f"Prompt:\n{formatted_prompt}")
    
#     inputs = tokenizer(formatted_prompt, return_tensors="pt").to("cuda")
#     output_tokens = model.generate(**inputs, max_new_tokens=250)
#     model_response = tokenizer.decode(output_tokens[0], skip_special_tokens=True)
    
#     cleaned_response = model_response.split("[/INST]")[-1].strip()
#     print(f"Model Response:\n{cleaned_response}")
#     print(f"True Label:\n{test_label}")

#     print(extract_prediction(cleaned_response))
#     print("\n#########################")

In [ ]:
# for i in range(10):
#     train_claim = dataset["train"]["statement"][i]
#     train_label = dataset["train"]["label"][i]
#     print(f"Claim:\n{train_claim}")
#     formatted_prompt = format_chat_prompt_v2(train_claim)
    
#     inputs = tokenizer(formatted_prompt, return_tensors="pt").to("cuda")
#     output_tokens = model.generate(**inputs, max_new_tokens=150)
#     model_response = tokenizer.decode(output_tokens[0], skip_special_tokens=True)
    
#     cleaned_response = model_response.split("[/INST]")[-1].strip()
#     print(f"Model Response:\n{cleaned_response}")
#     print(f"True Label:\n{train_label}")
    
#     print(extract_prediction(cleaned_response))
#     print("\n#########################")